# DataFrame의 다양한 응용
- 함수 Mapping
- 열 재구성
- Flitering
- 데이터 프레임 합치기 
- 그룹연산
- Multi Index 
- Pivot 

### 함수 Mapping 
- Series 또는 Dataframe의 Data를 특정함수에 일대일 대응 시키는 과정

In [2]:
import seaborn as sns

In [3]:
titanic = sns.load_dataset("titanic")

In [4]:
# titanic 데이터셋에서 age, fare 2개 열을 선택하여 데이터 프레임 만들기 
df = titanic[['age','fare']]

In [5]:
df['ten'] =10
df.head()

/var/folders/_z/5lt5f71x2fq6j9898gb_0m000000gn/T/ipykernel_32502/904097248.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ten'] =10


,age,fare,ten
0,22.0,7.2500,10
1,38.0,71.2833,10
2,26.0,7.9250,10
3,35.0,53.1000,10
4,35.0,8.0500,10


In [6]:
# 사용자 함수 정의 
# 10을 더하는 함수 
def add_10(n):
    return n + 10
# a + b 
def add_two_obj(a,b):
    return a + b

In [7]:
print(add_10(10))
print(add_two_obj(10,15))

20
25


In [8]:
# 시리즈 객체에 적용
# 나이에 + 10 
sr1 = df['age'].apply(add_10)
sr1.head()

0    32.0
1    48.0
2    36.0
3    45.0
4    45.0
Name: age, dtype: float64

In [9]:
# 시리즈 객체에 숫자 적용
sr2 = df['age'].apply(add_two_obj, b= 10)
# a = add_two_obj 
sr2.head()

0    32.0
1    48.0
2    36.0
3    45.0
4    45.0
Name: age, dtype: float64

---
### 람다함수 
- 시리즈 객체에 적용
- for문 같이 복잡한 식이 아닌 간단한 식에 적용 가능

In [10]:
sr3 = df['age'].apply(lambda x: x + 10 )
sr3.head()

0    32.0
1    48.0
2    36.0
3    45.0
4    45.0
Name: age, dtype: float64

In [11]:
# 더 많이 쓰는 보편적인 방법 
sr4 = df['age'].apply(lambda x: add_10(x) )
sr4.head()

0    32.0
1    48.0
2    36.0
3    45.0
4    45.0
Name: age, dtype: float64

In [12]:
# DataFrame 에 함수 Mapping
# applymap 모든 칼럼에 적용 
df_map = df.applymap(add_10)
df_map.head()

,age,fare,ten
0,32.0,17.2500,20
1,48.0,81.2833,20
2,36.0,17.9250,20
3,45.0,63.1000,20
4,45.0,18.0500,20


In [13]:
# 사용자 정의 함수 
# nan 값 찾기 
def missing_value(series):
    return series.isnull()

In [14]:
df.applymap(missing_value)

AttributeError: 'float' object has no attribute 'isnull'

In [ ]:
# type이 달라 하나씩 찾는 apply 가능 
df.apply(missing_value)

> apply는 column별(series)로 함수를 실행하나, applymap은 dataframe 전체로 함수 실행

In [ ]:
# DataFrame의 각 열의 최대값과 최소값의 차이를 출력
# 사용자 정의 
def min_max(series):
    return series.max() - series.min()


In [ ]:
result =df.apply(min_max)
print(result)

In [ ]:
# lambda 사용 
#df['add'] = df['age'].apply(lambda x: add_10(x) )
df['add'] = df.apply(lambda x: add_two_obj(x['age'],x['ten']),axis =1 )

df.head()

---
## pipe() 사용하기 

In [ ]:
df = titanic[['age','fare']]
df.head()

In [ ]:
# 각 열의 NaN 찾기
def missing_value(x):
    return x.isnull()

In [ ]:
# 각 열의 NaN 갯수 변환
def missing_count(x):
    return missing_value(x).sum()

In [ ]:
# 총 NaN 의 갯수 
def total_number_missing(x):
    return missing_count(x).sum()

In [ ]:
df.pipe(missing_value)

In [ ]:
df.pipe(missing_count)

In [ ]:
df.pipe(total_number_missing)

---
# 열 재구성

In [ ]:
# titanic 
df = titanic[['survived','pclass','sex','age']]
df.head()

In [ ]:
# 열 이름의 리스트 만들기
columns = list(df.columns)
columns
type(columns)

In [ ]:
df.columns

In [ ]:
# 열 이름을 알파벳 순으로 정렬
columns_sorted = sorted(columns)
columns_sorted 

In [ ]:
df[columns_sorted]

In [ ]:
# 열 이름을 기존순서와 정반대 순서로 정렬하기 
columns_sorted2 = reversed(columns)
df[columns_sorted2]

In [ ]:
columns_customed  = ['pclass','sex','age','survived']
columns_customed 

In [ ]:
df[columns_customed]

### 열분리 
- 하나의 열이 여러가지 정보를 담고 있을 때 각 정보를 서로 분리하여 사용 

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("../Data/주가데이터.xlsx")
df.head()

In [ ]:
df.info()

In [ ]:
# 년 , 월 , 일 데이터 분리하기 
df['연월일'] = df['연월일'].astype("str")
dates = df['연월일'].str.split('-')
dates.head()

In [ ]:
# year
dates[0][0]  


In [ ]:
df['연'] = dates.str.get(0)
df['월'] = dates.str.get(1)
df['일'] = dates.str.get(2)
df.head()

---
### Filering
- 특정 조건식을 만족하는 Data만 따로 추출하는 방법

In [ ]:
# 나이가 10대(10~19세)인 승객만 따로 선택
mask1 = (titanic.age >= 10)  & (titanic.age < 20)
df_teenage = titanic.loc[mask1, :]
df_teenage.head()

In [ ]:
# 나이가 10세(0~ 9) 미만이고 여성인 고객만 출력 
mask2 = (titanic.age < 9)  & (titanic.age >=  0)  & (titanic.sex =="female")
df_female_under10 = titanic.loc[mask2, :]
df_female_under10.head()


In [ ]:
# 나이가 10시 미만( 0 ~9세) 또는 60세 이상의 승객의 age,sex,alone열만 선택 

mask3 = (titanic['age'] < 10) | (titanic['age'] >= 60)
df_under10_moretan60 = titanic[mask3][['age','sex','alone']]
df_under10_moretan60.head()


### isin()

In [ ]:
isin_filter = titanic['sibsp'].isin([3,4,5])
titanic[isin_filter].head()

---
### Data Frame 병합 
- SQL의 Join과 비슷한 방식으로 어떤 기준에 의해 병합하는 개념
- 이때 기준이 되는 열이나 index를 Key라고 한다.
- key가 되는 열이나 index는 반드시 양쪽 DataFrame 에 존재해야 한다.

## 08번 DataFrame 정제 참고

In [ ]:
# 주식 데이터를 가져와서 데이터프레임 만들기 
df1 = pd.read_excel("../Data/stock price.xlsx")
df1

- id : 종목코드 
- stock_name : 회사이름
- vale : 시가총액
- price : 주가

In [ ]:
df2 = pd.read_excel("../Data/stock valuation.xlsx")
df2

- id : 종목코드 
- name : 회사이름
- eps : 주당순이익
- bps : 주당 순자산가치
- per : 주가 수익비율
- pbr : 주가 자산비율

In [ ]:
# 데이터프레임 합치기 - 교집합 
# id를 기준으로 합침 
merge_inner = pd.merge(df1, df2)
# default 값은 교집합
merge_inner

In [ ]:
# 데이터 프레임 합치기 - 합집합 
merger_outer = pd.merge(df1, df2 , how = 'outer')
merger_outer

In [ ]:
# 데이터 프레임 합치기 - 합집합 - 왼쪽 데이터 프레임 기준, 기준값은 회사명 
merger_left = pd.merge(df1, df2 , how = 'left', left_on = 'stock_name',right_on='name')
merger_left

In [ ]:
# 데이터 프레임 합치기 - 합집합 - 왼쪽 데이터 프레임 기준, 기준값은 회사명 
merger_right = pd.merge(df1, df2 , how = 'right', left_on = 'stock_name',right_on='name')
merger_right

In [ ]:
# 데이터 프레임 합치기 - 합집합 - 왼쪽 데이터 프레임 기준, 기준값은 회사명 
merger_left = pd.merge(df1, df2 , how = 'left', left_on = 'stock_name',right_on='name')
merger_left

In [ ]:
# df1에서 price가 50000 미만 인 데이터 찾기 
price= df1[df1['price'] < 50000]
price

In [ ]:
# df2에 없는 price 대신 id 기준 
value = pd.merge(price , df2)
value

In [ ]:
value.drop("name",axis=1,inplace=True)
value

---
# 그룹 연산 
- 복잡한 데이터를 어떤 기준에 따라 여러 그루으로 나눠서 관찰하는 것도 좋은 방법이다.
- 특정 기준을 적용하여 몇개의 그룹으로 분할하여 처리하는 과정이 그룹 연산이다.
- 1단계 : 분할 (split) => 데이터를 특정 조건에 의해 분할
- 2단계 : 적용 (apply) => 데이터를 집계, 변환, 필터링
- 3단계 : 결합 (combine) => 2단계 처리 결과를 하나로 결합

### 그룹 객체 만들기 (분할단계)

In [15]:
df = titanic.loc[:,['age','sex','class','fare','survived']]
df.head()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


In [ ]:
# class 열을 기준으로 분할
df['class'].unique()

In [16]:
# class 기준 그룹화
grouped = df.groupby(['class'])
list(grouped)
# first 
list(grouped)[0]

('First',
       age     sex  class     fare  survived
 1    38.0  female  First  71.2833         1
 3    35.0  female  First  53.1000         1
 6    54.0    male  First  51.8625         0
 11   58.0  female  First  26.5500         1
 23   28.0    male  First  35.5000         1
 ..    ...     ...    ...      ...       ...
 871  47.0  female  First  52.5542         1
 872  33.0    male  First   5.0000         0
 879  56.0  female  First  83.1583         1
 887  19.0  female  First  30.0000         1
 889  26.0    male  First  30.0000         1
 
 [216 rows x 5 columns])

In [17]:
# 그룹객체를 iteration 으로 출력 : 처음부터 5줄만 출력
for key, group in grouped:
    print("* key : ", key)
    print("* number : ", len(group))
    print(group.head())
    print("=" * 60)

* key :  First
* number :  216
     age     sex  class     fare  survived
1   38.0  female  First  71.2833         1
3   35.0  female  First  53.1000         1
6   54.0    male  First  51.8625         0
11  58.0  female  First  26.5500         1
23  28.0    male  First  35.5000         1
* key :  Second
* number :  184
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
20  35.0    male  Second  26.0000         0
21  34.0    male  Second  13.0000         1
* key :  Third
* number :  491
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0


In [18]:
# key를 가져옴 
g3 = grouped.get_group("Third").head()

In [19]:
g3

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
2,26.0,female,Third,7.9250,1
4,35.0,male,Third,8.0500,0
5,NaN,male,Third,8.4583,0
7,2.0,male,Third,21.0750,0


In [20]:
type(g3)

pandas.core.frame.DataFrame

In [21]:
# 연산 메소드 적용 
grouped.mean()
# class에 따라 sex object를 제외한 평균값 

,age,fare,survived
class,,,
First,38.233441,84.154687,0.629630
Second,29.877630,20.662183,0.472826
Third,25.140620,13.675550,0.242363


In [22]:
# class와 sex 기준으로 분할 
grouped_two = df.groupby(['class','sex'])
list(grouped_two)

[(('First', 'female'),
        age     sex  class      fare  survived
  1    38.0  female  First   71.2833         1
  3    35.0  female  First   53.1000         1
  11   58.0  female  First   26.5500         1
  31    NaN  female  First  146.5208         1
  52   49.0  female  First   76.7292         1
  ..    ...     ...    ...       ...       ...
  856  45.0  female  First  164.8667         1
  862  48.0  female  First   25.9292         1
  871  47.0  female  First   52.5542         1
  879  56.0  female  First   83.1583         1
  887  19.0  female  First   30.0000         1
  
  [94 rows x 5 columns]),
 (('First', 'male'),
        age   sex  class      fare  survived
  6    54.0  male  First   51.8625         0
  23   28.0  male  First   35.5000         1
  27   19.0  male  First  263.0000         0
  30   40.0  male  First   27.7208         0
  34   28.0  male  First   82.1708         0
  ..    ...   ...    ...       ...       ...
  839   NaN  male  First   29.7000         1
  8

In [23]:
for key, group in grouped_two:
    print("* key : ", key)
    print("* number : ", len(group))
    print(group.head())
    print("=" * 60)

* key :  ('First', 'female')
* number :  94
     age     sex  class      fare  survived
1   38.0  female  First   71.2833         1
3   35.0  female  First   53.1000         1
11  58.0  female  First   26.5500         1
31   NaN  female  First  146.5208         1
52  49.0  female  First   76.7292         1
* key :  ('First', 'male')
* number :  122
     age   sex  class      fare  survived
6   54.0  male  First   51.8625         0
23  28.0  male  First   35.5000         1
27  19.0  male  First  263.0000         0
30  40.0  male  First   27.7208         0
34  28.0  male  First   82.1708         0
* key :  ('Second', 'female')
* number :  76
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
41  27.0  female  Second  21.0000         0
43   3.0  female  Second  41.5792         1
53  29.0  female  Second  26.0000         1
* key :  ('Second', 'male')
* number :  108
     age   sex   class  fare  survived
17   

In [24]:
# 평균
grouped_two.mean()

age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447

In [25]:
# grouped_two에서 Third의 female만 group3f라는 dataframe 만들기
group3f = grouped_two.get_group(("Third","female"))
# tuple로 묶어서 key 값 그대로 가져오기 
group3f

,age,sex,class,fare,survived
2,26.0,female,Third,7.9250,1
8,27.0,female,Third,11.1333,1
10,4.0,female,Third,16.7000,1
14,14.0,female,Third,7.8542,0
18,31.0,female,Third,18.0000,0
...,...,...,...,...,...
863,NaN,female,Third,69.5500,0
875,15.0,female,Third,7.2250,1
882,22.0,female,Third,10.5167,0
885,39.0,female,Third,29.1250,0


---
### 그룹 연산 메소드 (적용 ~ 결합 )
- 집계 기능의 기본함수
- mean , max, min, sum, count, size,var, describe, info,first,last...

In [26]:
std_all = group.std
std_all

<bound method NDFrame._add_numeric_operations.<locals>.std of       age   sex  class     fare  survived
0    22.0  male  Third   7.2500         0
4    35.0  male  Third   8.0500         0
5     NaN  male  Third   8.4583         0
7     2.0  male  Third  21.0750         0
12   20.0  male  Third   8.0500         0
..    ...   ...    ...      ...       ...
877  19.0  male  Third   7.8958         0
878   NaN  male  Third   7.8958         0
881  33.0  male  Third   7.8958         0
884  25.0  male  Third   7.0500         0
890  32.0  male  Third   7.7500         0

[347 rows x 5 columns]>

In [27]:
grouped.fare.std()

class
First     78.380373
Second    13.417399
Third     11.778142
Name: fare, dtype: float64

In [28]:
def min_max(x):
    return x.max() - x.min()

In [29]:
grouped.agg(min_max)

/Users/tj/opt/anaconda3/lib/python3.9/site-packages/pandas/core/groupby/generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,age,fare,survived
class,,,
First,79.08,512.3292,1
Second,69.33,73.5000,1
Third,73.58,69.5500,1


In [31]:
grouped.agg(['min','max'])

age           sex       fare           survived    
         min   max     min   max  min       max      min max
class                                                       
First   0.92  80.0  female  male  0.0  512.3292        0   1
Second  0.67  70.0  female  male  0.0   73.5000        0   1
Third   0.42  74.0  female  male  0.0   69.5500        0   1

In [32]:
# column 을 key값으로 골라 원하는 value만 뽑기 
grouped.agg({'fare':['min','max'],'age':'mean'})

fare                  age
        min       max       mean
class                           
First   0.0  512.3292  38.233441
Second  0.0   73.5000  29.877630
Third   0.0   69.5500  25.140620